In [1]:
import geemap
!pip install pycrs

  Preparing metadata (setup.py) ... done
  Created wheel for pycrs: filename=PyCRS-1.0.2-py3-none-any.whl size=32687 sha256=0e511040b65eb789a1340ac7b884cfda2d5faf505f2b3d38c42aa069d08e706e
  Stored in directory: /root/.cache/pip/wheels/47/1d/70/7a5bdf33347e7c75e95b06b1fa38f076a59a9506653cc24aff
Successfully built pycrs


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import ee
ee. Authenticate()
ee.Initialize()

EEException: ee.Initialize: no project found. Call with project= or see http://goo.gle/ee-auth.

## Sentinel-2


In [ ]:
temporal = True
video = True
date1 = ee.Date('2023-09-01')
date2 = ee.Date('2023-10-01')
shapefile_path = "/content/drive/MyDrive/Research&Project/Cashew/Cashew_map_large-area/SEN/SEN_FIELD_TEAM/Polygons_GPS/shp/Tracé_003_LOU_001_COW_HVT.shp"
shapefile = geemap.shp_to_ee(shapefile_path) # FeatureCollection

Map = geemap.Map(basemap="Esri.WorldImagery")
S2collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterDate(date1, date2).filterBounds(shapefile)

vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}

if temporal == True:
  def add_image_to_map(image, Map):
    date = image.date().format('YYYY-MM-dd')
    name = f'Sentinel-2 Image: {date.getInfo()}'
    Map.addLayer(image, {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 3000,
        'gamma': 1.4
    }, name)

  image_list = S2collection.toList(S2collection.size())
  def get_image_date(image):
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    return ee.String('Sentinel-2 Image: ').cat(date)

  # Iterate through the list to add each image to the map
  for i in range(S2collection.size().getInfo()):
      image = ee.Image(image_list.get(i))
      layer_name = get_image_date(image).getInfo()
      Map.addLayer(image, vis_params, layer_name)

if temporal == False:
  image = S2collection.sort('CLOUDY_PIXEL_PERCENTAGE').first() # Sort the collection by cloud cover and get the least cloudy image
  if image is None:
    print('No images found for the specified date and location.')

  if image:
    Map.addLayer(image, vis_params, 'Sentinel-2 Image')

Map.addLayer(shapefile, {}, "shapefile")
Map.centerObject(shapefile, zoom=16)
Map

Map(center=[15.651070581897084, -16.165919348574533], controls=(WidgetControl(options=['position', 'transparen…

## Landsat8/9